# Lab note assistant

This notebook demonstrates a lab note assistant: Video-to-lab-note conversion using Vertex AI

Converting Videos-to-lab-note involves three steps: 
1. Protocol finder: Select protocol which best captures the step being performed in the video
2. Video comparing to ground-of-truth protocol → lab notes + errors in procedure
3. Analytics based on benchmark dataset: automatic comparison of errors found by lab note assistent vs actual errors

In this notebook, I will focus on the step two and three - Compare video with protocol.

In [1]:
from __future__ import annotations

# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

import configparser
import os
import sys
from pathlib import Path
import json
import pandas as pd
import pprint


from IPython.display import Markdown

import logging

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig, Part
from google.cloud import storage
from typing import TYPE_CHECKING, NamedTuple

import time
import datetime

path_to_append = Path(Path.cwd()).parent / "proteomics_specialist"
sys.path.append(str(path_to_append))
import video_to_protocol

config = configparser.ConfigParser()
config.read("../secrets.ini")

logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [2]:
config = configparser.ConfigParser()
config.read("../secrets.ini")

PROJECT_ID = config["DEFAULT"]["PROJECT_ID"]
vertexai.init(project=PROJECT_ID, location="us-central1")  # europe-west9 is Paris, europe-west3 is germany


In [3]:
os.environ["GOOGLE_CLOUD_PROJECT"] = config["DEFAULT"]["PROJECT_ID"]

storage_client = storage.Client()
bucket_name = "mannlab_videos"
bucket = storage_client.bucket(bucket_name)

In [ ]:
# test if model works

model = GenerativeModel("gemini-2.5-pro-preview-03-25")
model = GenerativeModel("gemini-2.5-flash-preview-04-17")
response = model.generate_content(
    ["test"],
)
response

candidates {
  content {
    role: "model"
    parts {
      text: "Okay, I\'m ready. What can I help you with?"
    }
  }
  finish_reason: STOP
  avg_logprobs: -11.424423217773438
}
usage_metadata {
  prompt_token_count: 1
  candidates_token_count: 14
  total_token_count: 429
  prompt_tokens_details {
    modality: TEXT
    token_count: 1
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 14
  }
}
model_version: "gemini-2.5-flash-preview-04-17"

In [5]:
def generate_content_from_model(
    inputs: Any,
    model_name: str = "gemini-2.5-pro-preview-03-25",
    temperature: float = 0.9,
) -> tuple:
    """Generate content using Google's Generative AI model.
    
    This function sends inputs to a specified Gemini model and returns the 
    generated response along with usage metadata.
    
    Parameters
    ----------
    inputs : Any
        The inputs to send to the model (text, images, or videos).
    model_name : str, default="gemini-2.5-pro-preview-03-25"
        Name of the generative model to use.
    temperature : float, default=0.9
        Controls the randomness of the output. Higher values (closer to 1.0)
        make output more random, lower values make it more deterministic.
        
    Returns
    -------
    tuple
        A tuple containing (response_text, usage_metadata)
        
    Raises
    ------
    ValueError
        If the model fails to generate content.
    """
    try:
        model = GenerativeModel(model_name)
        
        generation_config = GenerationConfig(
            temperature=temperature,
            # Uncomment if using single audio/video input
            # audio_timestamp=True
        )
        
        response = model.generate_content(
            inputs,
            generation_config=generation_config
        )
        lab_notes = response.text
        usage_metadata = response.usage_metadata
        
    except Exception as e:
        logger.exception("Error during content generation")
        raise ValueError(f"Failed to generate content: {str(e)}")
    
    return lab_notes, usage_metadata

In [6]:
def prepare_all_inputs(
    protocol_video_path: str,
    protocol_path: str,
    lab_video_path: str,
    lab_notes_path: str,
    bucket: str,
    prefix: str = "compare_protocol_video"
) -> dict:
    """Prepare all four standard inputs for the generative model.
    
    This function uploads the four standard files (lab video, protocol document, 
    lab notes video, and lab notes document) and formats them as inputs 
    for a generative model.
    
    Parameters
    ----------
    protocol_video_path : str
        Path to the file that shows the correct execution (ground truth) of the protocol.
    protocol_path : str
        Path to the protocol markdown file.
    lab_video_path : str
        Path to the lab video file.
    lab_notes_path : str
        Path to the lab notes markdown file.
    bucket : str
        GCS bucket name for uploading the files.
    prefix : str, default="compare_protocol_video"
        Prefix for the files in GCS bucket.
        
    Returns
    -------
    dict
        A dictionary containing the four formatted inputs:
        'protocol_video_input', 'protocol_input', 'lab_video_input', 'lab_notes_input'
    """
    
    video_uri = video_to_protocol.upload_video_to_gcs(protocol_video_path, bucket, prefix)
    file_extension = os.path.splitext(video_uri)[1].lower()[1:]
    protocol_video_input = [Part.from_uri(video_uri, mime_type=f"video/{file_extension}")]
    
    uri = video_to_protocol.upload_video_to_gcs(protocol_path, bucket, prefix)
    protocol_input = [Part.from_uri(uri, mime_type="text/md")]
    
    video_uri = video_to_protocol.upload_video_to_gcs(lab_video_path, bucket, prefix)
    lab_video_input = [Part.from_uri(video_uri, mime_type="video/mp4")]

    uri = video_to_protocol.upload_video_to_gcs(lab_notes_path, bucket, prefix)
    lab_notes_input = [Part.from_uri(uri, mime_type="text/md")]
    
    return {
        'protocol_video_input': protocol_video_input,
        'protocol_input': protocol_input,
        # 'protocol_input': 'not included',
        'lab_video_input': lab_video_input,
        'lab_notes_input': lab_notes_input
    }

In [7]:
def process_benchmark_dataset(csv_path, protocol_videos_base, lab_notes_videos_base, markdown_base, bucket, prefix):
    """
    Process the first two rows in the benchmark dataset CSV and prepare model inputs.
    
    Parameters:
    -----------
    csv_path : str
        Path to the CSV file containing benchmark dataset information
    protocol_videos_base : str
        Base path to the protocol videos directory
    lab_notes_videos_base : str
        Base path to the lab notes videos directory
    markdown_base : str
        Base path to the markdown files directory
    bucket : object
        The bucket object used in the prepare_all_inputs function
    prefix : str
        Prefix for the files in GCS bucket.
    
    Returns:
    --------
    dict
        Dictionary containing all model inputs for the first two rows in the CSV,
        with experiment names as keys
    """
    
    benchmark_df = pd.read_csv(
        csv_path, 
        sep=';'
    )
    
    all_model_inputs = {}
    
    for index, row in benchmark_df.head(2).iterrows(): # for testing .head(2).iterrows() or .iloc[[13, 14]] .iloc[::2]
        lab_video_path = os.path.join(protocol_videos_base, row["protocol video"])
        protocol_path = os.path.join(markdown_base, row["protocol"])
        lab_notes_video_path = os.path.join(lab_notes_videos_base, row["lab notes video"])
        lab_notes_path = os.path.join(markdown_base, row["lab notes"])
        
        dict_model_inputs = prepare_all_inputs(
            lab_video_path,
            protocol_path,
            lab_notes_video_path,
            lab_notes_path,
            bucket,
            prefix
        )
        dict_model_inputs['error_dict'] = row["error_dict"]
        
        experiment_name = row["lab notes"].split(".")[0]
        all_model_inputs[experiment_name] = dict_model_inputs
        
        print(f"Processed {experiment_name}")
        
        
    return all_model_inputs

In [8]:
def extract_errors(lab_notes, docu_steps, model_name="gemini-2.5-pro-preview-03-25", temperature=0.9):
    """
    Extract the identified errors of AI-generated lab notes.
    
    Parameters:
    -----------
    lab_notes : list
        The AI-generated lab notes to extract represented as a list of strings
    model_name : str, optional
        The model to use for evaluation, default is "gemini-2.5-pro-preview-03-25"
    temperature : float, optional
        Temperature setting for content generation, default is 0.9
        
    Returns:
    --------
    tuple
        A tuple containing (evaluation_text, usage_metadata)
    """
    prompt = """\
        # Instruction
        You are an expert evaluator. Your task is to extract the error positions and types of an AI-generated lab notes. It is most important to you to be exact in your response.

        # Rating rubics
        | Step | AI Response | AI Class |
        |------|-------------|----------------|
        | [Step] | No Error | N/A | 
        | [Step] | Error | Addition |
        | [Step] | Error | Deviation |
        | [Step] | Error | Omitted |
        | [Step] | Error | Error |
        | [Step] | Error | Deviation & Error |
        | [No Step] | N/A | N/A |

        # Output format
        | Step | AI Response | AI Class |
        |------|-------------|----------------|
        | 1 | No Error | N/A | 
        or
        | 2 | Error | [see above] | 

        # Your task
        Your task is to review the AI-generated lab notes for steps {docu_steps} and to create a table.
        """
    
    inputs = [prompt.format(docu_steps=docu_steps)  ] 
    inputs.extend(["## AI-Generated lab notes"])
    inputs.extend([lab_notes])
    inputs.extend(["## Output table"])

    evaluation, usage_metadata = generate_content_from_model(
        inputs,
        model_name=model_name,
        temperature=temperature,
    )
    # print(inputs)
    # print(evaluation)
    
    return evaluation, usage_metadata

# # Example
#         ## Example of an AI-generated lab_notes: 
#         "Alright, here is the lab_notes following your specifications:\n\n## lab_notes:# Change source: ESI source to UltraSource\n\n## Abstract\nThis protocol describes the procedure for switching from the ESI source to UltraSource.\n\n## Materials\n\n### Equipment\n- timsTOF Ultra Mass Spectrometer:\n  - ESI ion source\n  - UltraSource ion source \n- IonOpticks Column\n- Evosep One LC System with sample line\n- NanoViper Adapter (black)\n- Pliers\n\n## Procedure\n\n*Estimated timing: less than 10 minute*\n\n### Switch timsTOF to standby\n\n1. ✓ Verified the instrument was on standby mode\n2. ✓ Verified the syringe was inactive\n3. ✓ Selected 'CaptiveSpray' but did not activate it yet\n\n### Remove ESI source\n\n4. ✓ Disconnected the peak connector of the sample tubing\n5. ✓ Disconnected the nebulizer N₂ line\n6. ✓ Removed the source door. Hinged it out\n7. ❌ **Omitted:** Put on gloves after removing source door\n8. ✓ Removed the spray shield, and capillary cap.\n9. ⚠️ **Deviation:** Inspected the capillary position and gently pushed it back into proper position \n\n### Mount UltraSource\n\n10. ✓ Hinged the UltraSource door in and closed it \n11. ✓ Slid the UltraSource housing onto the source door and secured it by flipping the handles\n12. ✓ Connected the filter tubing to the source\n\n### Connect column and sample line\n\n13. ✓ Noted an IonOpticks column already inside UltraSource \n14. ✓ Noted the LC sample line had NanoViper adapter already attached\n15. ❌ **Omitted:** No need to snipp access liquid\n16. ✓ Held the column fititng of the IonOpticks column with a pliers.\n17. ✓ Hand-tightened the NanoViper of the LC sample line with the column fitting \n18. ✓ Drew the oven closer to the UltraSource, and secured it \n19. ✓ Removed the NanoViper adapter \n20. ✓ Placed the metal grounding screw\n21. ✓ Closed the lid of the oven\n22. ✓ Connected the oven to the electrical power supply\n23. ✓ Noted that with the correct temperature\n\n### Switch timsTOF to operate and idle flow\n\n24. ✓ Noted the CaptiveSpray function in timsControl had been activated.\n25. ✓ Noted that the instrument was on the operational mode\n26. ✓ Noted the idle flow was active\n27. ✓ Stay in timsControl\n28. ⚠️ **Deviation:** Checked the MS signal. Noted it needed to be adjusted to between 9-11 mbar\n\n## Expected Results\n- In timsControl, signal intensity should be above 10^7\n- Stable signal in in timsControl\n\n"

#         ## Example output table
#         | Step | AI Response | AI Class |
#         |------|-------------|----------------|
#         | 1 | No Error | N/A |
#         | 2 | No Error | N/A |
#         | 3 | No Error | N/A |
#         | 4 | No Error | N/A |
#         | 5 | No Error | N/A |
#         | 6 | No Error | N/A |
#         | 7 | Error | Omitted |
#         | 8 | No Error | N/A |
#         | 9 | Error | Deviation |
#         | 10 | No Error | N/A |
#         | 11 | No Error | N/A |
#         | 12 | No Error | N/A |
#         | 13 | No Error | N/A |
#         | 14 | No Error | N/A |
#         | 15 | Error | Omitted |
#         | 16 | No Error | N/A |
#         | 17 | No Error | N/A |
#         | 18 | No Error | N/A |
#         | 19 | No Error | N/A |
#         | 20 | No Error | N/A |
#         | 21 | No Error | N/A |
#         | 22 | No Error | N/A |
#         | 23 | No Error | N/A |
#         | 24 | No Error | N/A |
#         | 25 | No Error | N/A |
#         | 26 | No Error | N/A |
#         | 27 | No Error | N/A |
#         | 28 | Error | Deviation |
#         | 29 | N/A | Omitted |

def generate_lab_notes_evaluation(lab_notes_input, lab_notes, model_name="gemini-2.5-pro-preview-03-25", temperature=0.9):
    """
    Generate an evaluation of AI-generated lab notes against benchmark lab notes.
    
    Parameters:
    -----------
    lab_notes_input : list
        The benchmark lab notes (ground truth) represented as a list of strings
    lab_notes : list
        The AI-generated lab notes to evaluate represented as a list of strings
    model_name : str, optional
        The model to use for evaluation, default is "gemini-2.5-pro-preview-03-25"
    temperature : float, optional
        Temperature setting for content generation, default is 0.9
        
    Returns:
    --------
    tuple
        A tuple containing (evaluation_text, usage_metadata)
    """
    inputs = [
        """
        # Instruction
        You are an expert evaluator. Your task is to evaluate the lab notes quality of an AI-generated lab notes against a benchmark lab notes (ground truth). 

        # Evaluation Parts

        ## 5 Criteria:
        Evaluate the AI's lab notes quality based on these criteria:
        1. **Structure**: Did it keep only relevant sections: Aim, Materials, Procedure, Results?
        2. **Tense**: Did it use past tense to describe what actually happened, not what should happen?
        3. **Language**: Did it remove all instructional language and replace with observations?
        4. **Numbering**: Did it maintain step numbering of the original protocol even if order changed?
        5. **Timing**: Did it include exact actual timing, not estimated timing?

        ### Rating Rubric:
        For each criterion:
        - **Excellent**: The criterion was fully met with no issues
        - **Good**: The criterion was mostly met with minor issues
        - **Poor**: The criterion was not met or had significant issues

        # Output Format
        ## Lab notes Quality
        | Criterion | Rating | Explanation |
        |-----------|--------|-------------|
        | Structure | [Excellent/Good/Poor] | [Explanation] |
        | Tense | [Excellent/Good/Poor] | [Explanation] |
        | Language | [Excellent/Good/Poor] | [Explanation] |
        | Numbering | [Excellent/Good/Poor] | [Explanation] |
        | Timing | [Excellent/Good/Poor] | [Explanation] |

        # Evaluation Steps
        1. the lab notes quality of an AI-generated lab notes against a benchmark lab notes (ground truth) using the  5 criteria.
        2. Create a table summarizing the evaluation results.
        
        """
    ]
    inputs.extend(["""
        # Input Materials
        ## Benchmark Lab Notes (Ground Truth)
    """])
    inputs.extend(lab_notes_input)
    
    inputs.extend(["## AI-Generated Lab Notes"])
    inputs.extend([lab_notes])
    inputs.extend(["# Lab Notes Quality"])

    evaluation, usage_metadata = generate_content_from_model(
        inputs,
        model_name=model_name,
        temperature=temperature,
    )
    
    return evaluation, usage_metadata

def get_table_json_prompt(text_with_tables: str, table_identifier: str) -> str:
    """
    Generates a prompt to extract a specific table from text into JSON.

    Args:
        text_with_tables: The full text containing the table(s).
        table_identifier: A string to help the model identify the target table
                          (e.g., the table title, or a unique phrase near it).

    Returns:
        A formatted prompt string.
    """
    prompt = f"""
    You are an expert data extraction tool.
    Your task is to locate a specific table within the provided text and output its data as a JSON array.

    Here is the text containing the table(s):
    ---TEXT_START---
    {text_with_tables}
    ---TEXT_END---

    Identify the table that best matches the following title: "{table_identifier}"

    It is very important to you to output the data from ONLY this table as a valid JSON array. Each object in the array should represent a row from the table. The keys of each object should be the exact column headers from the identified table.

    Output Constraints:
    - Answer direct with the JSON.
    - If the specified table cannot be found, output an empty JSON array: []
    """
    return prompt


def extract_json_from_model_output(model_output_string):
    """
    Extract and parse JSON data from a model output string that contains JSON within code block markers.
    
    Parameters:
    -----------
    model_output_string : str
        The string output from the model that contains JSON within code block markers
        
    Returns:
    --------
    dataframe: A pandas DataFrame created from the JSON data, or None if extraction failed
    """
    start_marker = "```json"
    end_marker = "```"

    start_index = model_output_string.find(start_marker)
    end_index = model_output_string.find(end_marker, start_index + len(start_marker))  # Search for end marker after the start
    
    df = None
    if start_index != -1 and end_index != -1:
        extracted_json_string = model_output_string[start_index + len(start_marker):end_index].strip()
        
        try:
            json_data = json.loads(extracted_json_string)
            logger.info("Successfully extracted and parsed JSON.")
            
            if isinstance(json_data, list) and all(isinstance(item, dict) for item in json_data):
                df = pd.DataFrame(json_data)
            else:
                logger.warning("JSON data is not a list of dictionaries, could not create DataFrame.")
                
        except json.JSONDecodeError as e:
            logger.error(f"Error decoding JSON after extraction: {e}")
            logger.debug(f"Extracted string: {extracted_json_string}")
    else:
        logger.error("Could not find JSON code block markers in the output.")
        logger.debug(f"Model output: {model_output_string}")
    
    return df


def extract_table_to_dataframe(evaluation, table_name, model_name="gemini-2.5-pro-preview-03-25", temperature=0.9):
    """
    Extract a table from evaluation content and convert it to a DataFrame.
    
    Parameters:
    -----------
    evaluation : str
        The evaluation content containing tables
    table_name : str
        The name of the table to extract
    model_name : str, optional
        The model to use for content generation, default is "gemini-2.5-pro-preview-03-25"
    temperature : float, optional
        Temperature setting for content generation, default is 0.9
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame containing the extracted table data
    """
    extraction_prompt = get_table_json_prompt(evaluation, table_name)
    
    json_response, _ = generate_content_from_model(
        extraction_prompt,
        model_name=model_name,
        temperature=temperature
    )
    
    results_df = extract_json_from_model_output(json_response)
    
    return results_df


def identify_error_type(row):
    if row['Benchmark'] == 'No Error' and row['AI Response'] == 'No Error':
        return 'No Error (Correctly Identified)'
    elif row['Benchmark'] == 'Error' and row['AI Response'] == 'Error':
        return 'Error (Correctly Identified)'
    elif row['Benchmark'] == 'Error' and row['AI Response'] == 'No Error':
        return 'False Negative'
    elif row['Benchmark'] == 'No Error' and row['AI Response'] == 'Error':
        return 'False Positive'
    else:
        return 'Unknown' 


def classify_error_type(row):
    if row['Identification'] == 'Error (Correctly Identified)':
        if row['Class'] == row['AI Class']:
            return 'correct'
        else:
            return 'incorrect'
    else:
        return 'N/A' 
    

def generate_error_summary(df):
    """
    Generate a summary dictionary of error identification and classification statistics.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing error analysis results with 
                          'Benchmark', 'Identification', and 'Classification' columns
    
    Returns:
    dict: A nested dictionary containing error identification and classification statistics
    """
    total_steps = len(df)
    error_count = len(df[df['Benchmark'] == 'Error'])
    correctly_identified_errors = len(df[df['Identification'] == 'Error (Correctly Identified)'])
    false_negatives = len(df[df['Identification'] == 'False Negative'])
    false_positives = len(df[df['Identification'] == 'False Positive'])
    correct_identifications = len(df[(df['Identification'] == 'No Error (Correctly Identified)') | 
                                   (df['Identification'] == 'Error (Correctly Identified)')])

    total_errors_analyzed = len(df[df['Identification'] == 'Error (Correctly Identified)'])
    correctly_classified_errors = len(df[df['Classification'] == 'correct'])
    
    summary_dict = {
        'Error Identification Statistics': {
            'Steps evaluated': total_steps,
            'Correct identifications': correct_identifications,
            'Correct error identifications': correctly_identified_errors,
            'False negative count': false_negatives,
            'False positive count': false_positives,
        },
        'Error Classification Statistics': {
            'Total errors analyzed': total_errors_analyzed,
            'Correctly classified errors': correctly_classified_errors,
        }
    }
    
    return summary_dict


def process_and_evaluate_lab_notes(error_dict, lab_notes_gt, lab_notes_ai):
    """
    Process and evaluate lab notes by extracting errors, generating evaluations, 
    and creating summary statistics.
    
    Parameters:
    error_dict (list): List of error dictionaries
    lab_notes_gt (Any): Ground Truth lab notes to compare
    lab_notes_example (str): AI-generated lab notes to evaluate
    
    Returns:
    tuple: A tuple containing (valuation_response, df_errors, summary_dict)
    """
    error_dict = json.loads(error_dict)
    steps_list = [item["Step"] for item in error_dict]
    error_response, usage_metadata_extract_errors = extract_errors(lab_notes_ai, steps_list)

    evaluation_response, usage_metadata_semantic_eval = generate_lab_notes_evaluation(
        lab_notes_gt, lab_notes_ai)
    
    df_error_AI = extract_table_to_dataframe(error_response, "Table")
    df_error_AI["Step"] = df_error_AI["Step"].astype('float64')
    
    df_error_benchmark = pd.DataFrame(error_dict)
    df_errors = pd.merge(df_error_benchmark, df_error_AI, on='Step')

    df_errors['Identification'] = df_errors.apply(identify_error_type, axis=1)
    df_errors['Classification'] = df_errors.apply(classify_error_type, axis=1)
    
    summary_dict = generate_error_summary(df_errors)
    
    return evaluation_response, df_errors, summary_dict, usage_metadata_extract_errors, usage_metadata_semantic_eval

In [9]:
def generate_lab_notes_prompt(protocol_video_example, protocol_example, lab_video_example, lab_notes_example, 
                      protocol_video_input, protocol_input, lab_video_input, proteomics_knowledge, key,
                      model_name="gemini-2.5-pro-preview-03-25", temperature=0.9):
    """
    Generate corrected lab notes by comparing protocol with actual implementation.
    
    Parameters:
    -----------
    protocol_video_example : list
        Example protocol video content
    protocol_example : list
        Example protocol content
    lab_video_example : list
        Example lab video content
    lab_notes_example : list
        Example lab notes content
    protocol_video_input : list
        Input protocol video content to process
    protocol_input : list
        Input protocol content to process
    lab_video_input : list
        Input lab video content to process
    model_name : str, optional
        The model to use for generation, default is "gemini-2.5-pro-preview-03-25"
    temperature : float, optional
        Temperature parameter for generation, default is 0.9
        
    Returns:
    --------
    tuple
        A tuple containing the lab notes text and usage metadata
    """
    inputs = [
        """
        You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry. Your professional identity is defined by your ability to be exact in your responses and to produce meticulous, accurate results that others can trust completely.

        ## ====== Background Knowledge (FOR REFERENCE ONLY) ======
        [These documents are for building your proteomics background knowldge and are not part of today's task.]
        """]
    inputs.extend([proteomics_knowledge])
    inputs.extend([
        """
        # Instruction

        You work with following two inputs:
        - Ground truth written protocol: The official procedure description
        - Video to evaluate: The actual implementation by a researcher in a routine setting. Be aware that researchers tend to make mistakes in routine tasks.

        Compare the 'Ground truth written protocol' with the 'Video to evaluate', and create a "resulting lab notes" that reflects what actually happened in the 'video to evaluate'.
        

        # Evaluation

        ## Rating rubics for each step:
            1. It was followed correctly (no special notation needed)
            2. It was skipped: ❌ **Omitted:**
            3. It was carried out but wrongly: ❌ **Error:** (be specific about what happened)
            4. It was added: ➕ **Added:**
            5. It was carried out later in the procedure: ⚠️ **Deviation: Altered step order**
            6. A combination of 5. and the others: e.g. ⚠️ **Deviation: Altered step order** & ❌ **Omitted:**

        ## Follow this structured approach:

        * STEP 1: Read the 'Ground truth written protocol thoroughly and write it down again word-by-word.

        * STEP 2: Go through the 'Video to evaluate' completely from beginning to end.
            - Document all observed actions with timestamps
         
        Table 1:
        | Timestamp | Visual/Audio Action |\n
        |---|---|\n
        | [hh:mm:ss] |[Description of action] |\n
        | [hh:mm:ss] | [Description of action] |\n

        * STEP 3: Systematic comparison
            - Go through the 'Ground truth written protocol' as it would be a checklist step by step
            - For each step, specifically search for evidence in Table 1
            - If a step is not present, scan the entire Table 1 to confirm it wasn't performed out of sequence
            - For each step, clearly state:
                * Evaluate each step according to the rating rubics
                * The specific visual/audio evidence (or lack thereof) supporting your determination
                * Precise timestamps from the 'Video to evaluate'
            - If any step is present in Table 1 but not in 'Ground truth written protocol', then add this step in sequence and label it with the rating rubic '➕ **Added:**'
            
         
        Table 2:
        | Step | Step Description | Timestamp in 'Video to evaluate' | Comparison Result | Notes |\n
        |---|---|---|---|---|\n
        | 1 | [Description of step in 'Ground truth written protocol'] | [hh:mm:ss] | [Aligned/Partially/Misaligned] | [Explanation] |\n
        | 2 | [Description of step in 'Ground truth written protocol'] | [hh:mm:ss], [hh:mm:ss] | [Aligned/Partially/Misaligned] | [Explanation] |\n|

        * STEP 4: Create a "resulting lab notes" that accurately reflects what occurred in the 'Video to evaluate':
        - Rename sections as specified (Abstract to Aim, Expected Results to Results, Estimated timing to Timing)
        - Use past tense to describe actual observations
        - Include exact timing from the lab video
        - Remove instructional language and replace with observations
        - Omit Figures and References sections
                
        """
    ])
    
    inputs.extend(["""
        # ====== EXAMPLE (FOR REFERENCE ONLY) ======\n
        The following set of inputs and expected result should solely serve as an example and is not part of the evaluation task.\n
        """])
    inputs.extend(["## Example: 'Ground truth written protocol': \n"])
    inputs.extend(protocol_example)
    inputs.extend(["## Example: 'Video to evaluate': \n"])
    inputs.extend(lab_video_example)
    inputs.extend(["## Example - Expected result: 'resulting lab notes': \n"])
    inputs.extend(lab_notes_example)
    
    inputs.extend(["# ====== Beginn of EVALUATION TASK ====== \n"])
    inputs.extend(["## Important: The evaluation must be performed on the following video \n"])
    
    inputs.extend(["## Task: 'Ground truth written protocol': \n"])
    inputs.extend(protocol_input)
    inputs.extend(["## Task: 'Video to evaluate': \n"])
    inputs.extend(lab_video_input)
    inputs.extend([""" 
        As a reminder: Compare the 'Ground truth written protocol' against the 'video to evaluate' to retrieve the 'resulting lab notes'. Your final output should clearly state which rating rubic was identifyied for each step in the 'resulting lab notes'.
        """])
    # print(inputs)
    
    lab_notes, usage_metadata = generate_content_from_model(
        inputs,
        model_name=model_name,
        temperature=temperature,
    )
    
    return lab_notes, usage_metadata

In [11]:
csv_path = '/Users/patriciaskowronek/Documents/proteomics_specialist/data/benchmark_dataset.csv'
protocol_videos_base = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/protocols"
lab_notes_videos_base = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation"
markdown_base = "/Users/patriciaskowronek/Documents/proteomics_specialist/data"
prefix = "compare_protocol_video"

all_model_inputs = process_benchmark_dataset(csv_path, protocol_videos_base, lab_notes_videos_base, markdown_base, bucket, prefix)

Processed PlaceEvotips_docuCorrect
Processed PlaceEvotips_docuWrongPosition


In [ ]:
# analyze one specific video

subfolder_in_bucket = "knowledge"
path = "/Users/patriciaskowronek/Documents/proteomics_specialist/data/backgroundKnowledge.pdf"
file_uri = video_to_protocol.upload_video_to_gcs(
    path, bucket, subfolder_in_bucket
)
proteomics_knowledge = Part.from_uri(file_uri, mime_type="application/pdf")

example = 'Dilute_docuWrongVolume_PipettTipNotChanged'
example = 'PlaceEvotips_docuCorrect' # ToDo delete
protocol_video_example = all_model_inputs[example]['protocol_video_input']
protocol_example = all_model_inputs[example]['protocol_input']
lab_video_example = all_model_inputs[example]['lab_video_input']
lab_notes_example = all_model_inputs[example]['lab_notes_input']
copy_all_model_inputs = all_model_inputs.copy()
copy_all_model_inputs.pop(example)

items_list = list(copy_all_model_inputs.items())
key, value = items_list[0]
print(key)
            
lab_notes, usage_metadata = generate_lab_notes_prompt(
    protocol_video_example, protocol_example, lab_video_example, lab_notes_example,
    value['protocol_video_input'], value['protocol_input'], value['lab_video_input'], proteomics_knowledge, key,
    model_name="gemini-2.5-pro-preview-03-25", temperature=0.9
)
display(Markdown(lab_notes))

evaluation_response, df_errors, metrics, usage_metadata_extract_errors, usage_metadata_semantic_eval = process_and_evaluate_lab_notes(
    value['error_dict'], value['lab_notes_input'], lab_notes
)
display(Markdown(evaluation_response))
display(df_errors)
print(usage_metadata)
print('usage_metadata_extract_errors', usage_metadata_extract_errors)
print('usage_metadata_semantic_eval', usage_metadata_semantic_eval)


PlaceEvotips_docuWrongPosition


Okay, Professor Mann speaking. Precision is paramount in our field. Let us proceed with the meticulous evaluation of this laboratory procedure.

**STEP 1: Ground Truth Written Protocol (Rewritten Verbatim)**

# Placing Evotips in Evotip Boxes on the Evosep One System

## Abstract
This protocol describes the proper procedure for inspecting Evotips and placing Evotips in Evotip boxes on the liquid chromatography system Evosep One.

## Materials

### Equipment
- Evotips
  - Single-use stage tips for sample injection
  - Rack layout: Two columns (left and right)
  - Left column (top to bottom): S1, S2, S3
  - Right column (top to bottom): S4, S5, S6
  - Within each box: Standard 96-well format with A1 (top left), A12 (top right), H12 (bottom right)
- Evotip Boxes
  - 96-well format (A1-H12) (Figure 1)
- Evosep One System
  - Liquid chromatography system

### Reagents
- Formic acid (FA)
  ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.

### Reagent setup
- Buffer A: Consists of 0.1% (vol/vol) FA. The buffers are stable for at least 6 months at room temperature as long as they are protected from sunlight.

## Procedure
*Estimated timing: less than 1 minute*

1.  Verify that Evotip box is filled to a minimum depth of 1 cm with Buffer A solution.
2.  Place Evotip Box at S1 within the rack system of the Evosep instrument. Ensure each box is firmly seated in its designated position.
3.  Place an empty Evotip Box for Blank tips at S3. Ensure each box is firmly seated in its designated position.
4.  Inspect each Evotip before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it (Figure 2).
    **CRITICAL STEP**: Discard any Evotips showing signs of dryness or displaying a white-colored disc, as these conditions indicate compromised functionality that could affect sample analysis.
5.  Place the verified Evotips into the prepared Evotip boxes at S1 from A1 to A6.
6.  Place empty Evotips, called Blanks, at S3 from A1 to A6.
7.  Document the precise position of each placed Evotip.

## Expected Results
When the procedure is performed correctly, you should observe:
- Properly seated Evotip boxes in the rack system
- Visible Buffer A solution in boxes (1 cm depth)
- All non-blank Evotips showing pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip
- Accurate documentation of tip positions: Evotips that are placed at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.

## Figures

### Figure 1: Evosep positions
- Close-up of single Evotip box showing well positions (A1-H12)

### Figure 2: Evotip Quality Assessment
- Most Evotips: Properly hydrated Evotip with pale-colored disc and visible solvent
- Orange-highlighted Evotip: Compromised Evotip showing white/dry disc

## References
1.  Evosep One - User Guide: https://www.evosep.com/wp-content/uploads/2024/06/Evosep-One-User-Guide-v18.pdf
2.  Sample loading protocol for Evotips: https://www.evosep.com/wp-content/uploads/2020/03/Sample-loading-protocol.pdf

**STEP 2: Documentation of Observed Actions in 'Video to evaluate'**

**Table 1: Timestamped Actions**
| Timestamp     | Visual/Audio Action                                                                                             |
| :------------ | :-------------------------------------------------------------------------------------------------------------- |
| [00:00-00:03] | View of the Evosep One instrument. Researcher picks up the Evotip box intended for S1.                            |
| [00:03-00:09] | Researcher removes the yellow lid from the S1 box, visually inspects the liquid (Buffer A) level, replaces lid. |
| [00:10-00:14] | Researcher places the S1 Evotip box into position S1 on the Evosep rack, ensuring firm seating.                |
| [00:15-00:18] | Researcher picks up the empty Evotip box intended for S3 and places it into position S3, ensuring firm seating. |
| [00:21-00:22] | Researcher (Audio): "I want to place them from A1 to A6."                                                       |
| [00:22-00:31] | Researcher picks up Evotips (presumably containing sample) in pairs, briefly inspects, places into S1 box (A1-A6). |
| [00:31-00:33] | Researcher closes the lid of the source container for sample Evotips and sets it aside.                          |
| [00:36-00:47] | Researcher picks up empty Evotips (Blanks) in pairs, briefly inspects, places into S3 box (A1-A6).               |
| [00:47-00:49] | Researcher closes the lid of the source container for blank Evotips and sets it aside.                           |
| [00:50-00:55] | Final view of the Evosep rack with boxes S1 and S3 populated with Evotips (A1-A6 in both).                    |

**STEP 3: Systematic Comparison**

**Table 2: Protocol vs. Video Analysis**
| Step | Step Description                                                                                                                                                                                                                                                        | Timestamp in 'Video to evaluate' | Comparison Result             | Notes                                                                                                                                                                                                                                       |
| :--- | :---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------- | :---------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| 1    | Verify that Evotip box is filled to a minimum depth of 1 cm with Buffer A solution.                                                                                                                                                                                   | [00:03-00:09]                    | Aligned                       | The researcher visually inspected the liquid level in the S1 box after removing the lid. The depth appears adequate, consistent with the requirement.                                                                                     |
| 2    | Place Evotip Box at S1 within the rack system of the Evosep instrument. Ensure each box is firmly seated in its designated position.                                                                                                                                      | [00:10-00:14]                    | Aligned                       | The S1 box was placed correctly and pressed down, indicating firm seating.                                                                                                                                                                  |
| 3    | Place an empty Evotip Box for Blank tips at S3. Ensure each box is firmly seated in its designated position.                                                                                                                                                              | [00:15-00:18]                    | Aligned                       | The S3 box was placed correctly and pressed down, indicating firm seating.                                                                                                                                                                  |
| 4    | Inspect each Evotip before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it. **CRITICAL STEP**: Discard any Evotips showing signs of dryness or displaying a white-colored disc. | [00:22-00:31], [00:36-00:47]     | Aligned                       | The researcher performed a visual inspection while transferring the tips. The inspection was brief but appeared sufficient to confirm the pale disc and visible solvent, meeting the criteria. No tips were observed to be discarded.         |
|      |                                                                                                                                                                                                                                                                         | [00:21-00:22]                    | ➕ **Added:**                 | Researcher verbally stated the intended placement positions ("A1 to A6").                                                                                                                                                                   |
| 5    | Place the verified Evotips into the prepared Evotip boxes at S1 from A1 to A6.                                                                                                                                                                                          | [00:25-00:31]                    | Aligned                       | Six Evotips were placed sequentially into positions A1 through A6 in the S1 box.                                                                                                                                                            |
| 6    | Place empty Evotips, called Blanks, at S3 from A1 to A6.                                                                                                                                                                                                                | [00:39-00:47]                    | Aligned                       | Six empty Evotips (Blanks) were placed sequentially into positions A1 through A6 in the S3 box.                                                                                                                                           |
| 7    | Document the precise position of each placed Evotip.                                                                                                                                                                                                                    | Not observed                     | ❌ **Omitted:**               | The video concludes after tip placement; no action related to documentation (e.g., writing in a logbook, entering into software) was shown. This step, while crucial, was not performed within the timeframe captured by the video recording. |

**STEP 4: Resulting Lab Notes**

# Placing Evotips in Evotip Boxes on the Evosep One System

## Aim
This procedure documented the inspection and placement of Evotips (containing sample) and Blank Evotips into Evotip boxes on the Evosep One liquid chromatography system. The objective was to place six sample Evotips in box S1 (positions A1-A6) and six Blank Evotips in box S3 (positions A1-A6).

## Materials

### Equipment
- Evotips
- Evotip Boxes
- Evosep One System

### Reagent setup
- Buffer A: 0.1% (vol/vol) FA

## Procedure
*Timing: 0 minutes 55 seconds*

1.  **(00:03-00:09)** The Evotip box designated for S1 was verified to contain Buffer A solution to an adequate depth (visually estimated ≥ 1 cm).
2.  **(00:10-00:14)** The Evotip Box was placed at position S1 within the rack system of the Evosep instrument and firmly seated.
3.  **(00:15-00:18)** An empty Evotip Box for Blank tips was placed at position S3 and firmly seated.
4.  **(00:22-00:31, 00:36-00:47)** Each Evotip (sample and blank) was visually inspected during transfer prior to placement. All inspected tips displayed conditions consistent with proper preparation (pale-colored disc, visible solvent). No tips were discarded.
    *   ➕ **Added:** **(00:21-00:22)** The researcher verbally confirmed the intention to place tips from A1 to A6.
5.  **(00:25-00:31)** Six verified Evotips containing sample were placed into the Evotip box at S1, occupying positions A1 through A6.
6.  **(00:39-00:47)** Six empty Evotips (Blanks) were placed into the Evotip box at S3, occupying positions A1 through A6.
7.  ❌ **Omitted:** Documentation of the precise Evotip positions was not observed during the recorded procedure.

## Results
- Evotip boxes were properly seated in positions S1 and S3 on the Evosep rack system.
- The S1 Evotip box contained visible Buffer A solution, visually confirmed to be at an adequate depth.
- All placed Evotips (sample and blank) appeared correctly prepared based on visual inspection during transfer.
- Six sample Evotips were placed in S1 at positions A1-A6.
- Six Blank Evotips were placed in S3 at positions A1-A6.
- Documentation of tip placement was not performed within the scope of the video.

2025-05-05 18:56:50,243 - __main__ - INFO - Successfully extracted and parsed JSON.


## Lab notes Quality
| Criterion | Rating    | Explanation                                                                                                                                                              |
|-----------|-----------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Structure | Excellent | The AI notes retained only the essential sections: Aim, Materials, Procedure, and Results, matching the structure of the benchmark lab notes.                            |
| Tense     | Excellent | Past tense was correctly and consistently used throughout the Procedure and Results sections to describe the actions that were observed and documented (e.g., "was verified", "was placed", "were placed"). |
| Language  | Excellent | Instructional language from the original protocol (implied from the AI's Step 1) was removed and replaced with descriptive, observational language detailing what occurred (e.g., "visually inspected", "verbally confirmed"). |
| Numbering | Excellent | The AI maintained the original numbering (1-7) from the protocol it analyzed, mapping observed actions to these steps and clearly noting omissions (Step 7) and additions (verbal confirmation in Step 4). |
| Timing    | Excellent | The AI included precise, actual timing derived from its analysis (Overall: "0 minutes 55 seconds"; specific timestamps for actions within steps), rather than estimated timing. |

,Step,Benchmark,Class,AI Response,AI Class,Identification,Classification
0,1,No Error,N/A,No Error,N/A,No Error (Correctly Identified),N/A
1,2,No Error,N/A,No Error,N/A,No Error (Correctly Identified),N/A
2,3,No Error,N/A,No Error,N/A,No Error (Correctly Identified),N/A
3,4,Error,Omitted,No Error,N/A,False Negative,N/A
4,5,Error,Error,No Error,N/A,False Negative,N/A
5,6,No Error,N/A,No Error,N/A,No Error (Correctly Identified),N/A
6,7,No Error,N/A,No Error,N/A,No Error (Correctly Identified),N/A


prompt_token_count: 40895
candidates_token_count: 2672
total_token_count: 44423
prompt_tokens_details {
  modality: TEXT
  token_count: 2891
}
prompt_tokens_details {
  modality: AUDIO
  token_count: 2700
}
prompt_tokens_details {
  modality: VIDEO
  token_count: 28080
}
prompt_tokens_details {
  modality: DOCUMENT
  token_count: 7224
}
candidates_tokens_details {
  modality: TEXT
  token_count: 2672
}

prompt_token_count: 2943
candidates_token_count: 127
total_token_count: 3830
prompt_tokens_details {
  modality: TEXT
  token_count: 2943
}
candidates_tokens_details {
  modality: TEXT
  token_count: 127
}

prompt_token_count: 3496
candidates_token_count: 264
total_token_count: 5607
prompt_tokens_details {
  modality: TEXT
  token_count: 3496
}
candidates_tokens_details {
  modality: TEXT
  token_count: 264
}



In [ ]:
# analyze a sequence of videos

# Constants for retry logic
WAIT_TIME_BETWEEN_ITEMS = 10  # seconds
RETRY_WAIT_TIME = 120  # seconds
MAX_RETRIES = 3

CHECKPOINT_FILE = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/results/results_checkpoint.json"

def safe_json_dump(data, filename):
    """Handles non-serializable objects and converts items to strings"""
    def serialize(obj):
        if isinstance(obj, (dict)):
            return {k: serialize(v) for k, v in obj.items()}
        elif isinstance(obj, (list, tuple)):
            return [serialize(item) for item in obj]
        elif isinstance(obj, (int, float, str, bool)) or obj is None:
            return obj
        else:
            return str(obj)
    
    temp_file = f"{filename}.tmp"
    with open(temp_file, 'w') as f:
        json.dump(serialize(data), f)
    os.replace(temp_file, filename)

# Load checkpoint
results_collection = {}
last_processed_key = None
try:
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE) as f:
            data = json.load(f)
            results_collection = data.get('results', {})
            last_processed_key = data.get('last_key', None)
        print(f"Loaded checkpoint. Last processed key: {last_processed_key}")
except Exception as e:
    print(f"Error loading checkpoint: {e}")

# Upload knowledge files to Google Cloud Storage
subfolder_in_bucket = "knowledge"
path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/knowledge_base_selected/Connecting_or_disconnecting_column_2.pdf"
file_uri = video_to_protocol.upload_video_to_gcs(
    path, bucket, subfolder_in_bucket
)
proteomics_knowledge = Part.from_uri(file_uri, mime_type="application/pdf")

# example = 'ESIsourceToUltraSource_docuFogotOvenPowerSupply'
example = 'Dilute_docuWrongVolume_PipettTipNotChanged'
protocol_video_example = all_model_inputs[example]['protocol_video_input']
protocol_example = all_model_inputs[example]['protocol_input']
lab_video_example = all_model_inputs[example]['lab_video_input']
lab_notes_example = all_model_inputs[example]['lab_notes_input']
copy_all_model_inputs = all_model_inputs.copy()
copy_all_model_inputs.pop(example)

items_list = list(copy_all_model_inputs.items())
start_index = 0 if not last_processed_key else next((i + 1 for i, (k, _) in enumerate(items_list) if k == last_processed_key), 0)

for i in range(start_index, len(items_list)):
    key, value = items_list[i]
    
    for attempt in range(MAX_RETRIES):
        try:
            print(f"Processing {key} (attempt {attempt + 1})")
            
            start_generate_time = time.time()
            lab_notes, usage_metadata = generate_lab_notes_prompt(
                protocol_video_example, protocol_example, lab_video_example, lab_notes_example,
                value['protocol_video_input'], value['protocol_input'], value['lab_video_input'], proteomics_knowledge, key,
                model_name="gemini-2.5-pro-preview-03-25", temperature=0.9
            )
            end_generate_time = time.time()
            generate_time = end_generate_time - start_generate_time
            print(f"Time to generate lab notes: {generate_time:.2f} seconds")

            display(Markdown(lab_notes))
            
            start_evaluate_time = time.time()
            evaluation_response, df_errors, metrics, usage_metadata_extract_errors, usage_metadata_semantic_eval = process_and_evaluate_lab_notes(
                value['error_dict'], value['lab_notes_input'], lab_notes
            )
            end_evaluate_time = time.time()
            evaluate_time = end_evaluate_time - start_evaluate_time
            print(f"Time to process and evaluate lab notes: {evaluate_time:.2f} seconds")
            
            display(Markdown(evaluation_response))
            display(df_errors)
            
            # Store results
            results_collection[key] = {
                "inputs": {"experiment_name": key, **{k: v for k, v in value.items()}},
                "outputs": {
                    "lab_notes": lab_notes, 
                    "lab_notes_usage_metadata": usage_metadata,
                    "lab_notes_generate_time": generate_time,
                    "evaluation": evaluation_response, 
                    "eval_usage_metadata_extract_error": usage_metadata_extract_errors,
                    "eval_usage_metadata_semantic": usage_metadata_semantic_eval,
                    "eval_generate_time": evaluate_time,
                    "metrics": metrics
                }
            }
            
            safe_json_dump({"last_key": key, "results": results_collection}, CHECKPOINT_FILE)
            
            print(f"Waiting {WAIT_TIME_BETWEEN_ITEMS} seconds before next item...")
            time.sleep(WAIT_TIME_BETWEEN_ITEMS)
            break  # Success, exit retry loop
            
        except Exception as e:
            print(f"Error processing {key}: {e}")
            if attempt < MAX_RETRIES - 1:
                print(f"Waiting {RETRY_WAIT_TIME} seconds before retry...")
                time.sleep(RETRY_WAIT_TIME)
            else:
                print(f"Max retries reached for {key}, moving to next item")
                safe_json_dump({"last_key": key, "results": results_collection}, CHECKPOINT_FILE)

try:
    timestamp = time.time()
    safe_json_dump(results_collection, f"/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/results/results_checkpoint_{timestamp}.json")
    print("All processing complete. Final results saved.")
except Exception as e:
    print(f"Error saving final results: {e}")

Processing PlaceEvotips_docuCorrect (attempt 1)


Okay, Professor Mann speaking. Precision is paramount in proteomics, as it is in documenting our procedures. Let us meticulously analyze the provided materials.

## STEP 1: Ground Truth Written Protocol (Rewritten)

# Placing Evotips in Evotip Boxes on the Evosep One System

## Abstract
This protocol describes the proper procedure for inspecting Evotips and placing Evotips in Evotip boxes on the liquid chromatography system Evosep One.

## Materials

### Equipment
- Evotips
  - Single-use stage tips for sample injection
  - Rack layout: Two columns (left and right)
  - Left column (top to bottom): S1, S2, S3
  - Right column (top to bottom): S4, S5, S6
  - Within each box: Standard 96-well format with A1 (top left), A12 (top right), H12 (bottom right)
- Evotip Boxes
  - 96-well format (A1-H12) (Figure 1)
- Evosep One System
  - Liquid chromatography system

### Reagents
- Formic acid (FA)
  ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.

### Reagent setup
- Buffer A: Consists of 0.1% (vol/vol) FA. The buffers are stable for at least 6 months at room temperature as long as they are protected from sunlight.

## Procedure
*Estimated timing: less than 1 minute*

1.  Verify that Evotip box is filled to a minimum depth of 1 cm with Buffer A solution.
2.  Place Evotip Box at S1 within the rack system of the Evosep instrument. Ensure each box is firmly seated in its designated position.
3.  Place an empty Evotip Box for Blank tips at S3. Ensure each box is firmly seated in its designated position.
4.  Inspect each Evotip before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it (Figure 2).
    **CRITICAL STEP**: Discard any Evotips showing signs of dryness or displaying a white-colored disc, as these conditions indicate compromised functionality that could affect sample analysis.
5.  Place the verified Evotips into the prepared Evotip boxes at S1 from A1 to A6.
6.  Place empty Evotips, called Blanks, at S3 from A1 to A6.
7.  Document the precise position of each placed Evotip.

## Expected Results
When the procedure is performed correctly, you should observe:
- Properly seated Evotip boxes in the rack system
- Visible Buffer A solution in boxes (1 cm depth)
- All non-blank Evotips showing pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip
- Accurate documentation of tip positions: Evotips that are placed at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.

## Figures

### Figure 1: Evosep positions
- Close-up of single Evotip box showing well positions (A1-H12)

### Figure 2: Evotip Quality Assessment
- Most Evotips: Properly hydrated Evotip with pale-colored disc and visible solvent
- Orange-highlighted Evotip: Compromised Evotip showing white/dry disc

## References
1. Evosep One - User Guide: https://www.evosep.com/wp-content/uploads/2024/06/Evosep-One-User-Guide-v18.pdf
2. Sample loading protocol for Evotips: https://www.evosep.com/wp-content/uploads/2020/03/Sample-loading-protocol.pdf

## STEP 2: Video Observations

**Table 1:**
| Timestamp | Visual/Audio Action                                                                 |
| :-------- | :---------------------------------------------------------------------------------- |
| 00:00:00  | View of the Evosep One system with a Bruker timsTOF mass spectrometer next to it.    |
| 00:00:02  | Researcher's hands (wearing black gloves) enter the frame.                          |
| 00:00:03  | Researcher picks up a translucent Evotip box with a yellow grid top, labeled "SAMPLES". |
| 00:00:05  | Researcher holds the box, tilting it slightly, appearing to visually check the contents (liquid level). |
| 00:00:09  | Researcher places the Evotip box into the S1 position on the Evosep One rack.       |
| 00:00:12  | Researcher picks up a second, identical-looking Evotip box, labeled "BLANKS".       |
| 00:00:13  | Researcher places the second Evotip box into the S3 position on the Evosep One rack. |
| 00:00:15  | Researcher briefly adjusts the S3 box position.                                     |
| 00:00:17  | Researcher picks up a smaller clear plastic container holding Evotips.             |
| 00:00:20  | Researcher picks up two Evotips with tweezers from the container.                   |
| 00:00:21  | Researcher visually inspects the two tips.                                          |
| 00:00:22  | Researcher places the two tips into positions A1 and A2 of the Evotip box at S1.    |
| 00:00:23  | Researcher picks up two more Evotips from the container.                             |
| 00:00:24  | Researcher visually inspects the two tips.                                          |
| 00:00:25  | Researcher places the two tips into positions A3 and A5 of the Evotip box at S1 (appears to skip A4). |
| 00:00:27  | Researcher picks up two more Evotips from the container.                             |
| 00:00:28  | Researcher visually inspects the two tips.                                          |
| 00:00:29  | Researcher places the two tips into positions A6 and (presumably) A7 of the Evotip box at S1. *Correction*: Re-watching closely, the second tip is placed in A6. The first tip of this pair was placed in A5. So, A1, A2, A3, A5, A6 were populated. |
| 00:00:30  | Researcher puts down the container with remaining Evotips.                          |
| 00:00:33  | Researcher picks up another container, presumably with "Blank" Evotips.             |
| 00:00:36  | Researcher picks up two Evotips from this container.                               |
| 00:00:37  | Researcher visually inspects the two tips.                                          |
| 00:00:38  | Researcher places the two tips into positions A1 and A2 of the Evotip box at S3.    |
| 00:00:39  | Researcher picks up two more Evotips from the container.                             |
| 00:00:40  | Researcher visually inspects the two tips.                                          |
| 00:00:41  | Researcher places the two tips into positions A3 and A5 of the Evotip box at S3 (appears to skip A4). *Correction*: Re-watching closely, the placement is A3 and A5. |
| 00:00:41  | Researcher picks up two more Evotips from the container.                             |
| 00:00:41  | Researcher visually inspects the two tips.                                          |
| 00:00:41  | Researcher places the two tips into position A6 and (presumably) A7 of the Evotip box at S3. *Correction*: Re-watching closely, the second tip is placed in A6. The first tip of this pair was placed in A5. So, A1, A2, A3, A5, A6 were populated in S3. |
| 00:00:42  | Researcher puts down the container with remaining blank Evotips.                    |
| 00:00:45  | Researcher points sequentially at the placed tips in the S1 box (A1, A2, A3, A5, A6). |
| 00:00:48  | Researcher points sequentially at the placed tips in the S3 box (A1, A2, A3, A5, A6). |
| 00:00:51  | Researcher removes hands from the frame.                                            |
| 00:00:52  | End of relevant actions.                                                           |

## STEP 3: Systematic Comparison

**Table 2:**
| Step | Step Description                                                                                                                                                                                                                                                                                         | Timestamp in 'Video to evaluate' | Comparison Result | Notes                                                                                                                                                                                                                                                                                                                      |
| :--- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------- | :---------------- | :--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 1    | Verify that Evotip box is filled to a minimum depth of 1 cm with Buffer A solution.                                                                                                                                                                                                                      | 00:00:03 - 00:00:09              | Aligned           | The researcher picked up the Evotip box labeled "SAMPLES" and held it at an angle, allowing visual inspection of the liquid inside, consistent with the reference image for verifying solution level. The exact depth was not measured, but the visual check was performed. Assumed correct based on visual action. |
| 2    | Place Evotip Box at S1 within the rack system of the Evosep instrument. Ensure each box is firmly seated in its designated position.                                                                                                                                                                     | 00:00:09 - 00:00:11              | Aligned           | The Evotip box labeled "SAMPLES" was placed correctly into the S1 position.                                                                                                                                                                                                                                  |
| 3    | Place an empty Evotip Box for Blank tips at S3. Ensure each box is firmly seated in its designated position.                                                                                                                                                                                             | 00:00:12 - 00:00:15              | Aligned           | An Evotip box labeled "BLANKS" was placed correctly into the S3 position and adjusted for seating. It appeared empty of tips initially.                                                                                                                                                                    |
| 4    | Inspect each Evotip before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it (Figure 2). **CRITICAL STEP**: Discard any Evotips showing signs of dryness or displaying a white-colored disc...                   | 00:00:20 - 00:00:21, 00:00:23 - 00:00:24, 00:00:27 - 00:00:28, 00:00:36 - 00:00:37, 00:00:39 - 00:00:40, 00:00:41 | Aligned           | The researcher picked up Evotips (two at a time) and visually inspected them before placement in both S1 and S3 boxes, consistent with the protocol's requirement. No tips appeared to be discarded during this process in the video.                                                                             |
| 5    | Place the verified Evotips into the prepared Evotip boxes at S1 from A1 to A6.                                                                                                                                                                                                                           | 00:00:22, 00:00:25, 00:00:29     | Misaligned        | ❌ **Error:** Evotips were placed in positions A1, A2, A3, A5, and A6 in the S1 box. Position A4 was skipped. The protocol specifies placement from A1 to A6 consecutively.                                                                                                                                      |
| 6    | Place empty Evotips, called Blanks, at S3 from A1 to A6.                                                                                                                                                                                                                                                 | 00:00:38, 00:00:41               | Misaligned        | ❌ **Error:** Evotips (sourced from a container, presumably blanks) were placed in positions A1, A2, A3, A5, and A6 in the S3 box. Position A4 was skipped. The protocol specifies placement from A1 to A6 consecutively.                                                                                           |
| 7    | Document the precise position of each placed Evotip.                                                                                                                                                                                                                                                     | 00:00:45 - 00:00:51 (Pointing only) | Misaligned        | ❌ **Omitted:** The researcher pointed at the placed tips in S1 and S3 but did not perform any explicit documentation (e.g., writing, computer entry). Pointing is not equivalent to documentation as specified.                                                                                              |
| -    | *Pointing Confirmation*                                                                                                                                                                                                                                                                                  | 00:00:45 - 00:00:51              | Added             | ➕ **Added:** The researcher pointed sequentially at the tips placed in S1 (A1-A3, A5-A6) and then S3 (A1-A3, A5-A6), possibly as a final visual confirmation check, although this was not part of the written protocol.                                                                                      |

## STEP 4: Resulting Documentation

# Placing Evotips in Evotip Boxes on the Evosep One System

## Aim
This document describes the observed procedure for inspecting Evotips and placing them in Evotip boxes on the Evosep One liquid chromatography system, as performed in the evaluated video.

## Materials

### Equipment
- Evotips
  - Single-use stage tips for sample injection
  - Rack layout: Two columns (left and right)
  - Left column (top to bottom): S1, S2, S3
  - Right column (top to bottom): S4, S5, S6
  - Within each box: Standard 96-well format with A1 (top left), H12 (bottom right)
- Evotip Boxes (labeled "SAMPLES" and "BLANKS")
  - 96-well format (A1-H12)
- Evosep One System
  - Liquid chromatography system
- Tweezers

### Reagents
- Formic acid (FA)
  ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.

### Reagent setup
- Buffer A: Consisted of 0.1% (vol/vol) FA.

## Procedure
*Timing: Approximately 52 seconds*

1.  The researcher visually verified the Buffer A solution level in the Evotip box labeled "SAMPLES" [00:00:03 - 00:00:09].
2.  The Evotip Box labeled "SAMPLES" was placed at position S1 within the rack system of the Evosep instrument and appeared firmly seated [00:00:09 - 00:00:11].
3.  An Evotip Box labeled "BLANKS" (initially empty of tips) was placed at position S3 and adjusted to ensure it was firmly seated [00:00:12 - 00:00:15].
4.  Each Evotip was inspected visually (two at a time using tweezers) before placement to verify its condition [00:00:20 - 00:00:21, 00:00:23 - 00:00:24, 00:00:27 - 00:00:28, 00:00:36 - 00:00:37, 00:00:39 - 00:00:40, 00:00:41]. No tips were observed being discarded.
5.  ❌ **Error:** Verified Evotips were placed into the Evotip box at S1 in positions A1, A2, A3, A5, and A6. Position A4 was skipped [00:00:22, 00:00:25, 00:00:29]. (Protocol specified A1-A6).
6.  ❌ **Error:** Evotips designated as Blanks were placed into the Evotip box at S3 in positions A1, A2, A3, A5, and A6. Position A4 was skipped [00:00:38, 00:00:41]. (Protocol specified A1-A6).
7.  ❌ **Omitted:** The precise position of each placed Evotip was not documented.
7.1 ➕ **Added:** The researcher pointed sequentially at the placed tips in S1 and S3 as a visual confirmation step [00:00:45 - 00:00:51].

## Results
- Evotip boxes were observed properly seated in the S1 and S3 positions of the rack system.
- Visible Buffer A solution was observed in the S1 box prior to placement.
- All placed Evotips appeared to have been visually inspected prior to placement; none were observed to be discarded.
- The documentation of tip positions differed from the expected results: Evotips were placed at S1 (positions A1, A2, A3, A5, A6) and S3 (positions A1, A2, A3, A5, A6). Position A4 was skipped in both boxes.
- No formal documentation step was observed.

2025-05-04 04:18:27,250 - __main__ - INFO - Successfully extracted and parsed JSON.


## Documentation Quality
| Criterion | Rating    | Explanation                                                                                                                               |
|-----------|-----------|-------------------------------------------------------------------------------------------------------------------------------------------|
| Structure | Excellent | The AI documentation (STEP 4) maintained the relevant sections: Aim, Materials, Procedure, and Results, matching the benchmark structure. |
| Tense     | Excellent | The AI documentation consistently used the past tense (e.g., "verified", "was placed", "was inspected") to describe the observed actions. |
| Language  | Excellent | The AI documentation replaced instructional language with observations of the actions performed (e.g., "The researcher visually verified..."). |
| Numbering | Excellent | The AI maintained the original protocol's step numbering (1-7) when describing the observed actions, adding a sub-step (7.1) for clarity.   |
| Timing    | Excellent | The AI documentation included the specific timing observed from the video ("Approximately 52 seconds"), replacing the estimated timing.    |

,Step,Benchmark,Class,AI Response,AI Class,Identification,Classification
0,1,No Error,N/A,No Error,N/A,No Error (Correctly Identified),N/A
1,2,No Error,N/A,No Error,N/A,No Error (Correctly Identified),N/A
2,3,No Error,N/A,No Error,N/A,No Error (Correctly Identified),N/A
3,4,No Error,N/A,No Error,N/A,No Error (Correctly Identified),N/A
4,5,No Error,N/A,No Error,N/A,No Error (Correctly Identified),N/A
5,6,No Error,N/A,No Error,N/A,No Error (Correctly Identified),N/A
6,7,No Error,N/A,No Error,N/A,No Error (Correctly Identified),N/A


...

Okay, Professor Mann speaking. Precision is paramount in our field. Let us meticulously analyze the provided documentation and the visual record of the procedure.

## Evaluation Output

**STEP 1: Ground truth written protocol (transcribed verbatim)**

# Dispensing Protocol

## Abstract
This protocol describes how to spin Evotips.

## Materials

### Equipment
- Evotips
- Evotip holders wilth solvent reservior
- Centrifuge

## Procedure
*Estimated timing: 1 minute*

1.  Equaly distribute Evotips onto two Evotip holders. For instance 12 Evotips each. Do this either manually or by using a multichannel pipette.
2.  Switch on the centrifuge.
3.  Place the Evotip holders within centrifuge holders at opposite positions.
4.  Spin the Evotips at 800g for 1 minute in a centrifuge.

## Expected Results
A balanced centrifuge that spins at 800g for 1 minute.

**STEP 2: Observed Actions in 'Video to evaluate' (Table 1)**

| Timestamp   | Visual/Audio Action                                                                                                                                        |
| :---------- | :--------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 00:00:00    | Researcher commenced the procedure with two yellow-topped Evotip boxes.                                                                                     |
| 00:00:00    | Researcher utilized a multichannel pipette to transfer material (presumed to be Evotips) from one Evotip box to the other.                                   |
| 00:06:00    | Pipetting action concluded. Pipette was set aside.                                                                                                         |
| 00:07:00    | Researcher closed the lid of the source Evotip box.                                                                                                        |
| 00:08:00    | Researcher picked up both Evotip boxes.                                                                                                                    |
| 00:10:00    | Researcher stated (audio): "Then it should be spun at 800g for one minute."                                                                                |
| 00:12:00    | Researcher transported the Evotip boxes towards the centrifuge area.                                                                                       |
| 00:18:00    | Researcher placed each Evotip box into a designated white centrifuge holder.                                                                               |
| 00:21:00    | Researcher interacted with the centrifuge lid, which was already in the open position.                                                                     |
| 00:22:00    | Researcher positioned the two holders containing the Evotip boxes into the centrifuge rotor, ensuring they were placed at diametrically opposite locations. |
| 00:25:00    | Researcher interacted with the centrifuge's touchscreen control panel.                                                                                     |
| 00:26:00    | Parameters were adjusted on the touchscreen: Speed/force set to '800' (unit assumed to be 'g'). Time set to '00:01:00'.                                      |
| 00:29:00    | Settings were confirmed/saved on the touchscreen interface.                                                                                                |
| 00:31:00    | Researcher closed the centrifuge lid securely.                                                                                                             |
| 00:33:00    | Researcher initiated the centrifuge run by pressing 'Start' on the control panel.                                                                          |
| 00:34:00    | Centrifuge commenced operation. The display showed the timer counting down from 00:01:00 and the g-force increasing towards the setpoint of 800g.          |
| 00:39:00    | Display indicated the centrifuge reached 800g.                                                                                                             |
| 00:50:00    | Video recording concluded while the centrifuge was still operating and the timer was counting down (approximately 00:00:10 remaining).                   |

**STEP 3: Systematic Comparison (Table 2)**

| Step | Step Description                                                                                                     | Timestamp in 'Video to evaluate' | Comparison Result                                       | Notes                                                                                                                                                                                                                                     |
| :--- | :------------------------------------------------------------------------------------------------------------------- | :------------------------------- | :------------------------------------------------------ | :---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 1    | Equally distribute Evotips onto two Evotip holders. For instance 12 Evotips each. Do this either manually or by using a multichannel pipette. | 00:00:00 - 00:06:00              | Aligned                                                 | Action performed using a multichannel pipette as described. Visual confirmation of "Evotips" or the precise number distributed (e.g., 12 each) is not possible from the video angle, but the action aligns with the instruction.        |
| 1.1  | *(Implied Action)* Close source Evotip box lid.                                                                      | 00:07:00                         | ➕ **Added:**                                         | Researcher closed the lid of the box presumably used as the source. This is a logical tidiness step not explicitly mentioned in the protocol.                                                                                        |
| 1.2  | *(Implied Action)* State intended parameters.                                                                        | 00:10:00 - 00:12:00              | ➕ **Added:**                                         | Researcher verbally confirmed the intended spin parameters ("spun at 800g for one minute"). This vocalization is not part of the written protocol.                                                                                  |
| 2    | Switch on the centrifuge.                                                                                            | Not Observed                     | ❌ **Omitted:**                                         | The centrifuge display was already illuminated and active when the researcher approached (00:21:00), indicating it was already powered on prior to the sequence shown. No action of switching the centrifuge on was observed in the video. |
| 3    | Place the Evotip holders within centrifuge holders at opposite positions.                                             | 00:18:00 - 00:24:00              | Aligned                                                 | The researcher first placed the Evotip boxes (holders) into white centrifuge adapters (00:18:00) and subsequently placed these assemblies into the rotor at opposite positions (00:22:00-00:24:00), correctly ensuring balance.       |
| 3.1  | *(Implied Action)* Open centrifuge lid.                                                                              | 00:21:00                         | ➕ **Added:**                                         | Researcher interacted with the lid handle. Although the lid was already open, this action is typically required. The protocol implicitly requires opening but doesn't list it.                                                          |
| 4    | Spin the Evotips at 800g for 1 minute in a centrifuge.                                                               | 00:25:00 - 00:50:00              | Aligned (Process initiated, completion not shown) | The process of spinning was correctly initiated. Parameters (800g, 1 min) were set (see 4.1), lid closed (see 4.2), and run started (see 4.3). Centrifuge reached 800g. Video ends before the full minute elapsed.                   |
| 4.1  | *(Implied Action)* Set centrifuge parameters.                                                                        | 00:25:00 - 00:29:00              | ➕ **Added:**                                         | Researcher used the touchscreen interface to explicitly set the speed to 800 (unit assumed g) and time to 1 minute. This operational detail is not specified in the written protocol.                                                  |
| 4.2  | *(Implied Action)* Close centrifuge lid.                                                                             | 00:31:00                         | ➕ **Added:**                                         | Researcher closed the centrifuge lid before starting the run. This is a necessary safety and operational step, implied but not explicitly listed in the protocol.                                                                  |
| 4.3  | *(Implied Action)* Start centrifuge run.                                                                             | 00:33:00                         | ➕ **Added:**                                         | Researcher pressed the 'Start' button on the control panel to begin the centrifugation. This action is implied but not explicitly listed in the protocol.                                                                           |

**STEP 4: Resulting documentation**

# Dispensing Protocol - Observed Procedure

## Aim
This documentation details the observed procedure for spinning Evotips in a centrifuge, based on video analysis.

## Materials

### Equipment Observed
-   Evotip boxes (yellow-topped containers acting as holders)
-   Multichannel pipette
-   White centrifuge holders/adapters
-   Thermo Scientific Multifuge X Pro Series Centrifuge (or similar model with touchscreen)

## Procedure
*Timing: Approximately 1 minute for the full centrifuge run (distribution time variable, observed run initiated, not completed within video duration)*

1.  Material (presumed Evotips) was distributed between two yellow-topped Evotip boxes using a multichannel pipette (00:00:00 - 00:06:00).
    *   1.1 ➕ **Added:** The lid of the source Evotip box was closed (00:07:00).
    *   1.2 ➕ **Added:** The researcher verbally stated the intended centrifuge parameters: "spun at 800g for one minute" (00:10:00).
2.  ❌ **Omitted:** The action of switching on the centrifuge was not observed; the instrument was already powered on.
3.  The Evotip boxes were placed into white centrifuge holders (00:18:00), and these assemblies were then placed into the centrifuge rotor at opposite positions (00:22:00 - 00:24:00).
    *   3.1 ➕ **Added:** The researcher interacted with the centrifuge lid, which was already open (00:21:00).
4.  The centrifuge run was initiated to spin the material at 800g for 1 minute.
    *   4.1 ➕ **Added:** Centrifuge parameters were set using the touchscreen: speed/force to 800 (unit presumed 'g'), time to 00:01:00 (00:25:00 - 00:29:00).
    *   4.2 ➕ **Added:** The centrifuge lid was closed (00:31:00).
    *   4.3 ➕ **Added:** The centrifuge run was started via the control panel (00:33:00). The centrifuge was observed accelerating and reaching the target 800g. The timer began counting down from 1 minute. The video concluded before the run was finished (00:34:00 - 00:50:00).

## Results
-   Two Evotip boxes were placed in adapters and loaded symmetrically into the centrifuge.
-   A centrifuge run was correctly programmed for 800g for 1 minute and initiated.
-   The centrifuge was observed operating at 800g. The completion of the 1-minute spin was not recorded in the provided video segment.

2025-05-04 07:00:42,367 - __main__ - INFO - Successfully extracted and parsed JSON.


## Documentation Quality
| Criterion | Rating    | Explanation                                                                                                                                                              |
|-----------|-----------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Structure | Excellent | The AI documentation correctly includes only the relevant sections: Aim, Materials, Procedure, and Results, mirroring the structure of the benchmark.                     |
| Tense     | Excellent | The AI consistently used the past tense (e.g., "was distributed," "were placed," "was initiated") to describe the observed actions, as required for reporting what happened. |
| Language  | Excellent | Instructional language was successfully removed and replaced with objective observational statements (e.g., "Material... was distributed," "The centrifuge run was initiated"). |
| Numbering | Excellent | The AI maintained the original step numbering (1-4) from the inferred original protocol (seen in AI Step 1) and appropriately used sub-numbering and notes (➕ Added, ❌ Omitted) for observed deviations, similar to the benchmark's method. |
| Timing    | Excellent | The AI included specific observed timings using timestamps from the video analysis (e.g., "(00:00:00 - 00:06:00)") and accurately reported the actual observed run duration in the procedure header, replacing estimated timing. |

,Step,Benchmark,Class,AI Response,AI Class,Identification,Classification
0,1,Error,Error,No Error,N/A,False Negative,N/A
1,2,Error,Omitted,No Error,N/A,False Negative,N/A
2,3,No Error,N/A,No Error,N/A,No Error (Correctly Identified),N/A
3,4,Error,Error,No Error,N/A,False Negative,N/A


Waiting 10 seconds before next item...
All processing complete. Final results saved.


In [ ]:
def load_json_file(filename):
    """Load the JSON file that was saved with safe_json_dump"""
    with open(filename, 'r') as f:
        return json.load(f)

def flatten_dict(nested_dict, prefix=''):
    flattened = {}
    for key, value in nested_dict.items():
        if isinstance(value, dict):
            flattened.update(flatten_dict(value, f"{prefix}{key}_"))
        else:
            flattened[f"{prefix}{key}"] = value
    return flattened

loaded_data = load_json_file("/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/results/20250504_results_gemini_2.0-flash.json")
    
flattened_data = [flatten_dict(data) for data in loaded_data.values()]
df = pd.DataFrame(flattened_data)
df_subset = df[['inputs_experiment_name',
    'outputs_metrics_Error Identification Statistics_Steps evaluated',
       'outputs_metrics_Error Identification Statistics_Correct identifications',
       'outputs_metrics_Error Identification Statistics_Correct error identifications'
       'outputs_metrics_Error Identification Statistics_False positive count',
       'outputs_metrics_Error Identification Statistics_False negative count',
       'outputs_metrics_Error Classification Statistics_Total errors analyzed',
       'outputs_metrics_Error Classification Statistics_Correctly classified errors',]]

new_columns = ['experiment_name', 'Steps evaluated',
        'Correct identifications',  'Correct error identifications', 
       'False positive count', 'False negative count',
       'Errors analyzed', 'Correctly classified errors',
       ]

# ToDo: Correct table entries where model was hallouzinating 
# df_with_summary_stats.loc[df_with_summary_stats['experiment_name'] == 'ESIsourceToUltraSource_docuFogotOvenPowerSupply', 
#     ['Correct identifications', 'False positive count', 'False negative count', 'Correct error identifications']] = [20, 9, 0, 0]
# df_with_summary_stats

# ToDo: Calculate summary statistics after table is corrected
# precision = correct_identifications / total_steps if correct_identifications > 0 else 0
# recall = correctly_identified_errors / error_count if error_count > 0 else 0
# classification_accuracy = correctly_classified_errors / total_errors_analyzed if total_errors_analyzed > 0 else 0

# 'Identification precision', 'Error recall rate', 'Classification accuracy'

df_subset.columns = new_columns
df_subset = df_subset.replace('N/A', 0)

summary_stats = pd.Series({
    'experiment_name': 'Summary',
    'Steps evaluated': df_subset['Steps evaluated'].sum(),
    'Correct identifications': df_subset['Correct identifications'].sum(),
    'Identification accuracy': df_subset['Identification accuracy'].mean(),
    'Error recall rate': df_subset['Error recall rate'].mean(),
    'False positive count': df_subset['False positive count'].sum(),
    'False negative count': df_subset['False negative count'].sum(),
    'Errors analyzed': df_subset['Errors analyzed'].sum(),
    'Correctly classified errors': df_subset['Correctly classified errors'].sum(),
    'Classification accuracy': df_subset['Classification accuracy'].mean()
})

df_with_summary_stats = pd.concat([df_subset, pd.DataFrame([summary_stats])], ignore_index=True)
df_with_summary_stats

,experiment_name,Steps evaluated,Correct identifications,Identification accuracy,Error recall rate,False positive count,False negative count,Errors analyzed,Correctly classified errors,Classification accuracy
0,PlaceEvotips_docuCorrect,7,4,0.571429,0.000000,3,0,0,0,0.000000
1,PlaceEvotips_docuWrongPosition,7,3,0.428571,0.000000,3,1,0,0,0.000000
2,PlaceEvotips_docuLiquidNotChecked,7,5,0.714286,0.000000,0,2,0,0,0.000000
3,PlaceEvotips_docuBoxAngeled,7,1,0.142857,0.000000,5,1,0,0,0.000000
4,ConnectingColumnSampleLine_docuWithoutStandbyA...,14,12,0.857143,0.800000,1,1,4,4,1.000000
5,ESIsourceToUltraSource_docuCorrect,29,20,0.689655,0.000000,9,0,0,0,0.000000
6,ESIsourceToUltraSource_docuFogotOvenPowerSupply,29,22,0.758621,0.000000,0,7,0,0,0.000000
7,UltraSourceToESIsource_docuCorrect,25,22,0.880000,0.000000,3,0,0,0,0.000000
8,UltraSourceToESIsource_docuForgotN2Line,25,20,0.800000,1.000000,5,0,1,1,1.000000
9,UltraSourceToESIsource_docuForgotGlovesANDCapi...,25,18,0.720000,0.500000,6,1,1,1,1.000000


In [ ]:
process_and_evaluate_lab_notes(
                value['error_dict'], value['lab_notes_input'], lab_notes_example
            )

2025-05-01 09:34:05,727 - __main__ - ERROR - Error during content generation
Traceback (most recent call last):
  File "/var/folders/54/g1_1ycl12hl02xj_g_nm_6cm0000gn/T/ipykernel_56368/3333729111.py", line 48, in generate_content_from_model
    response = model.generate_content(
  File "/Users/patriciaskowronek/Conda/miniconda3/envs/docu_test/lib/python3.9/site-packages/vertexai/generative_models/_generative_models.py", line 695, in generate_content
    return self._generate_content(
  File "/Users/patriciaskowronek/Conda/miniconda3/envs/docu_test/lib/python3.9/site-packages/vertexai/generative_models/_generative_models.py", line 812, in _generate_content
    request = self._prepare_request(
  File "/Users/patriciaskowronek/Conda/miniconda3/envs/docu_test/lib/python3.9/site-packages/vertexai/generative_models/_generative_models.py", line 3387, in _prepare_request
    request_v1beta1 = super()._prepare_request(
  File "/Users/patriciaskowronek/Conda/miniconda3/envs/docu_test/lib/python3

ValueError: Failed to generate content: Unexpected item type: [file_data {
  mime_type: "text/md"
  file_uri: "gs://mannlab_videos/compare_protocol_video/Dilute_docuWrongVolume_PipettTipNotChanged.md"
}
].Only types that represent a single Content or a single Part are supported here.

In [ ]:
lab_notes_example = "Alright, here is the documentation following your specifications:\n\n## Documentation:# Change source: ESI source to UltraSource\n\n## Abstract\nThis protocol describes the procedure for switching from the ESI source to UltraSource.\n\n## Materials\n\n### Equipment\n- timsTOF Ultra Mass Spectrometer:\n  - ESI ion source\n  - UltraSource ion source \n- IonOpticks Column\n- Evosep One LC System with sample line\n- NanoViper Adapter (black)\n- Pliers\n\n## Procedure\n\n*Estimated timing: less than 10 minute*\n\n### Switch timsTOF to standby\n\n1. ✓ Verified the instrument was on standby mode\n2. ✓ Verified the syringe was inactive\n3. ✓ Selected 'CaptiveSpray' but did not activate it yet\n\n### Remove ESI source\n\n4. ✓ Disconnected the peak connector of the sample tubing\n5. ✓ Disconnected the nebulizer N₂ line\n6. ✓ Removed the source door. Hinged it out\n7. ❌ **Omitted:** Put on gloves after removing source door\n8. ✓ Removed the spray shield, and capillary cap.\n9. ⚠️ **Deviation:** Inspected the capillary position and gently pushed it back into proper position \n\n### Mount UltraSource\n\n10. ✓ Hinged the UltraSource door in and closed it \n11. ✓ Slid the UltraSource housing onto the source door and secured it by flipping the handles\n12. ✓ Connected the filter tubing to the source\n\n### Connect column and sample line\n\n13. ✓ Noted an IonOpticks column already inside UltraSource \n14. ✓ Noted the LC sample line had NanoViper adapter already attached\n15. ❌ **Omitted:** No need to snipp access liquid\n16. ✓ Held the column fititng of the IonOpticks column with a pliers.\n17. ✓ Hand-tightened the NanoViper of the LC sample line with the column fitting \n18. ✓ Drew the oven closer to the UltraSource, and secured it \n19. ✓ Removed the NanoViper adapter \n20. ✓ Placed the metal grounding screw\n21. ✓ Closed the lid of the oven\n22. ✓ Connected the oven to the electrical power supply\n23. ✓ Noted that with the correct temperature\n\n### Switch timsTOF to operate and idle flow\n\n24. ✓ Noted the CaptiveSpray function in timsControl had been activated.\n25. ✓ Noted that the instrument was on the operational mode\n26. ✓ Noted the idle flow was active\n27. ✓ Stay in timsControl\n28. ⚠️ **Deviation:** Checked the MS signal. Noted it needed to be adjusted to between 9-11 mbar\n\n## Expected Results\n- In timsControl, signal intensity should be above 10^7\n- Stable signal in in timsControl\n\n"

display(Markdown(lab_notes_example))

Alright, here is the documentation following your specifications:

## Documentation:# Change source: ESI source to UltraSource

## Abstract
This protocol describes the procedure for switching from the ESI source to UltraSource.

## Materials

### Equipment
- timsTOF Ultra Mass Spectrometer:
  - ESI ion source
  - UltraSource ion source 
- IonOpticks Column
- Evosep One LC System with sample line
- NanoViper Adapter (black)
- Pliers

## Procedure

*Estimated timing: less than 10 minute*

### Switch timsTOF to standby

1. ✓ Verified the instrument was on standby mode
2. ✓ Verified the syringe was inactive
3. ✓ Selected 'CaptiveSpray' but did not activate it yet

### Remove ESI source

4. ✓ Disconnected the peak connector of the sample tubing
5. ✓ Disconnected the nebulizer N₂ line
6. ✓ Removed the source door. Hinged it out
7. ❌ **Omitted:** Put on gloves after removing source door
8. ✓ Removed the spray shield, and capillary cap.
9. ⚠️ **Deviation:** Inspected the capillary position and gently pushed it back into proper position 

### Mount UltraSource

10. ✓ Hinged the UltraSource door in and closed it 
11. ✓ Slid the UltraSource housing onto the source door and secured it by flipping the handles
12. ✓ Connected the filter tubing to the source

### Connect column and sample line

13. ✓ Noted an IonOpticks column already inside UltraSource 
14. ✓ Noted the LC sample line had NanoViper adapter already attached
15. ❌ **Omitted:** No need to snipp access liquid
16. ✓ Held the column fititng of the IonOpticks column with a pliers.
17. ✓ Hand-tightened the NanoViper of the LC sample line with the column fitting 
18. ✓ Drew the oven closer to the UltraSource, and secured it 
19. ✓ Removed the NanoViper adapter 
20. ✓ Placed the metal grounding screw
21. ✓ Closed the lid of the oven
22. ✓ Connected the oven to the electrical power supply
23. ✓ Noted that with the correct temperature

### Switch timsTOF to operate and idle flow

24. ✓ Noted the CaptiveSpray function in timsControl had been activated.
25. ✓ Noted that the instrument was on the operational mode
26. ✓ Noted the idle flow was active
27. ✓ Stay in timsControl
28. ⚠️ **Deviation:** Checked the MS signal. Noted it needed to be adjusted to between 9-11 mbar

## Expected Results
- In timsControl, signal intensity should be above 10^7
- Stable signal in in timsControl



In [ ]:
# Usefull helper function

def check_file_exists(file_path):
    if os.path.exists(file_path):
        print(f"File found: {file_path}")
    else:
        print(f"Error: File not found: {file_path}")